[Follow the steps outline in documentation](https://docs.google.com/document/d/1lq8ZlseA99eZe0Y3AeNxqE0ix5TNQf5WARBZ9YSHLAM/edit?tab=t.0)

- one at a time : #N13E103 #'N10E105','S01W063'

In [1]:
import logging
from utilenames import tilenames_mkd, tilenames_tls, tilenames_rgn
from os.path import join, isfile
from os import makedirs
from glob import glob
from uinterp import riofill
from ufuncs import get_raster_info, gdal_regrid
from uvars import gdsm_v_fn, gdtm_v_fn, egm08_fn, outdir, indir
import sys
from uvars import topoxcale_dir
sys.path.append(topoxcale_dir)
from topoxcale.mlxcale import mldownxcale
from topoxcale.sagaxcale import gwrdownxcale
import time

In [2]:
# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("processing.log"), # add date to this 
        logging.StreamHandler(sys.stdout)
    ]
)

ti = time.perf_counter()
#tilenames #N13E103 #'N10E105','S01W063'
logging.info('Loading main variables')
tilenames = ['N13E103']#tilenames_mkd + tilenames_tls + tilenames_rgn
varname = "tdem_dem" # use EDEM #deps: filling methos for my DEMs
roitilenames = tilenames
makedirs(outdir, exist_ok=True)
xres, yres = 0.01057350068885258912, -0.01057350068885258912
t_epsg = 'EPSG:4326'
mode = "num"

varpath = f"{indir}/*/*{varname}.tif"
varfiles = glob(varpath)
roivarfiles = [fi for fi in varfiles for t in roitilenames if t in fi]

ta = time.perf_counter()
gdsmf_tiles = []
gdtmf_tiles = []
geoid_tiles = []
logging.info(f'setting baseline dem as {varname}')

2025-04-29 16:22:04,596 - INFO - Loading main variables
2025-04-29 16:22:04,605 - INFO - setting baseline dem as tdem_dem


In [3]:
si = 0 
for fi in roivarfiles:
    tilename = fi.split('/')[-2]
    tile_dir = join(outdir, 'TILES', tilename)
    makedirs(tile_dir, exist_ok=True)
    logging.info(f"Processing tile: {tilename}")
    _, _, _, xmin, xmax, ymin, ymax, _, _ = get_raster_info(fi)
    
    gdsmv_tile = f"{tile_dir}/{tilename}_gdsm_void.tif"
    gdtmv_tile = f"{tile_dir}/{tilename}_gdtm_void.tif"
    geoid_tile = f"{tile_dir}/{tilename}_egm08.tif"
    logging.info(f'regriding GDSM @{tilename}...')
    gdal_regrid(gdtm_v_fn, gdtmv_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)
    logging.info(f'regriding GDTM  @{tilename}...')
    gdal_regrid(gdsm_v_fn, gdsmv_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)

    logging.info(f'regriding GEOID  @{tilename}...')
    gdal_regrid(egm08_fn, geoid_tile, xmin, ymin, xmax, ymax, xres, yres, mode, t_epsg, overwrite=False)

    gdsmf_tile = gdsmv_tile.replace('void.tif', f'riofill{si}.tif')
    gdtmf_tile = gdtmv_tile.replace('void.tif', f'riofill{si}.tif')

    logging.info(f'tFilling GDSM @{tilename}...')
    riofill(gdtmv_tile, gdtmf_tile, si=si)
    logging.info(f'tFilling GDSM @{tilename}...')
    riofill(gdsmv_tile, gdsmf_tile, si=si)
    
    gdsmf_tiles.append(gdsmf_tile)
    gdtmf_tiles.append(gdtmf_tile)
    geoid_tiles.append(geoid_tile)

tb = time.perf_counter()
logging.info(f'Void filling time: {(tb - ta)/60:.2f} min')

2025-04-29 16:22:04,622 - INFO - Processing tile: N13E103
2025-04-29 16:22:04,636 - INFO - regriding GDSM @N13E103...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
Creating output file that is 95P x 96L.
Processing /media/ljp238/12TBWolf/ARCHIEVE/GEDI/GRID/comprexn/GEDI_L3_be/GEDI03_elev_lowestmode_mean_2019108_2022019_002_03_EPSG4326.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
2025-04-29 16:22:04,730 - INFO - regriding GDTM  @N13E103...
Source NoData Value: -9999.0
Destination NoData Value: -9999.0
Creating output file that is 95P x 96L.
Processing /media/ljp238/12TBWolf/ARCHIEVE/GEDI/GRID/comprexn/GEDI_L3_vh/GEDI03_rh100_mean_2019108_2022019_002_03_EPSG4326.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
2025-04-29 16:22:04,817 - INFO - regriding GEOID  @N13E103...
Source NoData Value: -32767.0
Destination NoData Value: -9999.0
Creating output file that is 95P x 96L.
Processing /media/ljp238/12TBWolf/ARCHIEVE/GEOI

transform geoid

In [4]:
from ugeoid import ellipsoid2orthometric

In [5]:
# egm_files = glob(f"{indir}/*/*egm08.tif")
# egm_tile = [fi for fi in egm_files for t in roitilenames if t in fi][0]
# egm_tile

In [11]:
gdsm_tile = gdsmf_tiles[0]
gdsm_tile_egm = gdsm_tile.replace('.tif', '_egm.tif')

gdtm_tile = gdtmf_tiles[0]
gdtm_tile_egm = gdtm_tile.replace('.tif', '_egm.tif')

geoid_tile = geoid_tiles[0]

ellipsoid2orthometric(gdsm_tile,geoid_tile,gdsm_tile_egm)
ellipsoid2orthometric(gdtm_tile,geoid_tile,gdtm_tile_egm)

extract dod

In [12]:
import numpy as np
import rasterio

def calculate_dod(dem1_path, dem2_path, output_path=None):
    """
    Calculate the Difference of DEMs (DoD) by subtracting dem1 from dem2.

    Parameters:
    - dem1_path (str): Path to the first DEM (baseline or older).
    - dem2_path (str): Path to the second DEM (newer or comparison).
    - output_path (str, optional): Path to save the output DoD raster. If None, doesn't save.

    Returns:
    - dod_array (np.ndarray): The difference array (dem2 - dem1).
    """
    with rasterio.open(dem1_path) as src1, rasterio.open(dem2_path) as src2:
        if src1.shape != src2.shape or src1.transform != src2.transform or src1.crs != src2.crs:
            raise ValueError("Input DEMs must have the same shape, transform, and CRS.")

        dem1 = src1.read(1).astype(np.float32)
        dem2 = src2.read(1).astype(np.float32)

        # Mask nodata values
        mask = (dem1 == src1.nodata) | (dem2 == src2.nodata)
        dod = dem2 - dem1
        dod[mask] = np.nan

        if output_path:
            profile = src1.profile
            profile.update(dtype='float32', nodata=np.nan)
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(dod, 1)

    return dod


In [15]:
gdsm_tile = gdsmf_tiles[0]
gdsm_tile_egm = gdsm_tile.replace('.tif', '_egm.tif')
gdtm_tile = gdtmf_tiles[0]
gdtm_tile_egm = gdtm_tile.replace('.tif', '_egm.tif')
dod_tile = gdsm_tile.replace('gdsm_riofill0', 'dod')
dod_tile_egm = gdsm_tile_egm.replace('gdsm_riofill0', 'dod')

In [16]:

dod_tile, dod_tile_egm

('/media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N13E103/N13E103_dod.tif',
 '/media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N13E103/N13E103_dod_egm.tif')

In [18]:
_ = calculate_dod(gdtm_tile_egm, gdsm_tile_egm, dod_tile_egm)
_ = calculate_dod(gdtm_tile, gdsm_tile, dod_tile)

downxcale_gwr with edem

we have done this with 1K regrid, try doing with 12 regrid

In [ ]:
varpath = f"{indir}/*/*{varname}.tif"
varfiles = glob(varpath)
roivarfiles = [fi for fi in varfiles for t in roitilenames if t in fi]


In [10]:
gdtmf_tiles,gdsmf_tiles

(['/media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N13E103/N13E103_gdtm_riofill.tif'],
 ['/media/ljp238/12TBWolf/BRCHIEVE/GDEM/TILES/N13E103/N13E103_gdsm_riofill.tif'])

'/media/ljp238/12TBWolf/BRCHIEVE/TILES12/N13E103/N13E103_egm08.tif'